# (Iterative) Hirshfeld method

In this tutorial, we will introduce how to use the `horton_part` API method to execute (iterative) Hirshfeld partitioning methods.

First, we import the required libraries.

In [ ]:
import logging

import numpy as np
from gbasis.evals.eval import evaluate_basis
from gbasis.wrappers import from_iodata
from grid import AtomGrid, ExpRTransform, UniformInteger
from iodata import load_one
from setup import prepare_grid_and_dens, print_results

from horton_part import HirshfeldIWPart, HirshfeldWPart, ProAtomDB, ProAtomRecord
from horton_part.scripts.generate_cube import (
    _setup_cube_grid,
    prepare_input_cube,
    to_cube,
)

np.set_printoptions(precision=3, linewidth=np.inf, suppress=True)

In (iterative) Hirshfeld methods, a database of atomic radial density profiles is required. Therefore, different single-atom DFT or other *ab initio* calculations should be executed. Here, we use the Gaussian package. The checkpoint files, in `fchk` format, are generated using Gaussian tools as well.

Next, the radial atomic density is obtained by computing the spherical average of the atomic density. This process can be applied using the `gbasis` and `grid` packages.

In [ ]:
def prepare_record(filename):
    """Prepare molecular grid and density."""
    mol = load_one(filename)

    # Specify the integration grid
    rtf = ExpRTransform(5e-4, 2e1, 120 - 1)
    uniform_grid = UniformInteger(120)
    rgrid = rtf.transform_1d_grid(uniform_grid)

    # Get the spin-summed density matrix
    one_rdm = mol.one_rdms.get("post_scf", mol.one_rdms.get("scf"))
    basis = from_iodata(mol)
    assert len(mol.atnums) == 1
    grid = AtomGrid.from_preset(atnum=mol.atnums[0], preset="fine", rgrid=rgrid)

    basis_grid = evaluate_basis(basis, grid.points)
    rho = np.einsum("ab,bp,ap->p", one_rdm, basis_grid, basis_grid, optimize=True)
    spline = grid.spherical_average(rho)
    radial_rho = spline(rgrid.points)

    record = ProAtomRecord(mol.atnums[0], mol.charge, mol.energy, rgrid, radial_rho)
    return record

In this tutorial, we focus on the water molecule and therefore only need data for hydrogen and oxygen atoms. It should be noted that data for both cations and anions are also used.

In [ ]:
h_1 = prepare_record("./data/atoms/001__h_001_q+00/mult02/atom.fchk")
h_2 = prepare_record("./data/atoms/001__h_002_q-01/mult01/atom.fchk")
o_7 = prepare_record("./data/atoms/008__o_007_q+01/mult04/atom.fchk")
o_8 = prepare_record("./data/atoms/008__o_008_q+00/mult03/atom.fchk")
o_9 = prepare_record("./data/atoms/008__o_009_q-01/mult02/atom.fchk")

records = [h_1, h_2, o_7, o_8, o_9]
db = ProAtomDB(records)

Next, we prepare the molecular grid and densities as described in the `Basic Usage` section. It should be noted that the radial grids for atoms in the water molecule should match the atomic radial grids used for the corresponding single atoms in the atomic database calculations.

In [ ]:
def prepare_argument_dict(mol, grid, rho, db):
    """Prepare basic input arguments for all AIM methods."""
    kwargs = {
        "coordinates": mol.atcoords,
        "numbers": np.asarray(mol.atnums, np.int64),
        "pseudo_numbers": mol.atnums,
        "grid": grid,
        "moldens": rho,
        "lmax": 3,
        "proatomdb": db,
    }
    return kwargs

## Hirshfeld method

The Hirshfeld method can be executed using the `HirshfeldWPart` class.

In [ ]:
mol, grid, rho = prepare_grid_and_dens("data/h2o.fchk")
h = HirshfeldWPart(**prepare_argument_dict(mol, grid, rho, db))
h.do_all()
print_results(h)

## Iterative Hirshfeld method

The iterative Hirshfeld method can be executed using `HirshfeldIWpart` class.

In [ ]:
hi = HirshfeldIWPart(**prepare_argument_dict(mol, grid, rho, db))
hi.do_all()
print_results(hi)

## Export AIM density to a cube file

First, a new cube grid must be constructed and the molecular density evaluated at the grid points.
Next, the AIM density is recalculated on this cube grid using the AIM weight functions obtained from the partitioning method (e.g., `h` or `hi`).


In [ ]:
def update_pro(part, iatom, proatmdens, grid):
    """Update propars.

    Parameters
    ----------
    iatom : int
        The index of the atom for which the pro-atom densities are to be updated.
    proatdens : 1D np.ndarray
        The array representing the pro-atom density. This array is updated with the new density values
        for the specified atom.
    grid : Grid
        A `Grid` object, e.g., Molecular grid or a 3D cubic grid.
    """
    work = np.zeros((grid.size,))
    part.eval_proatom(iatom, work, grid)
    proatmdens += work
    proatmdens += 1e-100


def compute_aimdens(part, grid, moldens):
    """See ``Compute AIM density.

    Parameters
    ----------
    part : HirshfeldWPart or HirshfeldIWPart.
        A partitioning method.
    grid : Grid
        A `Grid` object, e.g., Molecular grid or a 3D cubic grid.
    moldens : 1D np.ndarray
        Molecular density on `grid` points.
    """
    proatmdens = np.zeros((part.natom, grid.size))
    for iatom in range(part.natom):
        update_pro(part, iatom, proatmdens[iatom, :], grid)
    promoldens = np.sum(proatmdens, axis=0)

    at_weights = np.zeros((part.natom, grid.size))
    # Compute the atomic weights by taking the ratios: pro-atom-dens/pro-molecule-dens.
    for iatom in range(part.natom):
        at_weights[iatom, :] = proatmdens[iatom, :] / promoldens
    np.clip(at_weights, 0, 1, out=at_weights)
    aimdens = at_weights * moldens[None, :]
    return proatmdens, promoldens, at_weights, aimdens

In [ ]:
logger = logging.getLogger("__main__")
# A denser grid points should be applied to reproduce correct population.
cube_grid = _setup_cube_grid(mol.atnums, mol.atcoords, logger, spacing=0.2)
cube_grid, data = prepare_input_cube(mol, 1000, False, False, logger, cube_grid)
cube_moldens = data["density"]
proatmdens, promoldens, at_weights, aimdens = compute_aimdens(
    hi, cube_grid, cube_moldens
)

sum_pop = 0.0
for iatom in range(hi.natom):
    pop = cube_grid.integrate(aimdens[iatom, :])
    print(f"The population of atom {iatom} using cube_grid is : {pop:.3f}")
    sum_pop += pop
    # export AIM density of each atom in water to a cube file
    to_cube(
        f"h2o_aimrho_{iatom}.cube",
        mol.atnums,
        mol.atcorenums,
        mol.atcoords,
        cube_grid,
        aimdens[iatom, :],
    )
    # export pro-atom density to a cube file
    to_cube(
        f"h2o_prorho_{iatom}.cube",
        mol.atnums,
        mol.atcorenums,
        mol.atcoords,
        cube_grid,
        proatmdens[iatom, :],
    )

# Note: the pupulation of sum of all atoms is not equal to 10 due to a coarse cube grid used.
print(f"The total population using cube_grid is : {sum_pop:.3f}")
print(
    f"The total population using partitioning grid is : {hi.grid.integrate(hi._moldens):.3f}"
)
# export molecular density of water to cube file
to_cube(
    "h2o_molrho.cube",
    mol.atnums,
    mol.atcorenums,
    mol.atcoords,
    cube_grid,
    cube_moldens,
)
# export pro-molecular density of water to cube file
to_cube(
    "h2o_promolrho.cube",
    mol.atnums,
    mol.atcorenums,
    mol.atcoords,
    cube_grid,
    cube_moldens,
)

By plotting these cube files with a visualization program such as VESTA (using an isosurface value of 0.05), one obtains the following:

| ![](h2o_molrho.png) |  ![](h2o_promolrho.png) |
|----------------------|---|
| Molecular density    |  Pro-molecular density   |

| ![](h2o_aimrho_0.png) | ![](h2o_prorho_0.png) |
|------------------------|------------------------|
| AIM density (O)        | Pro-atom density (O)   |

| ![](h2o_aimrho_1.png) | ![](h2o_prorho_1.png) |
|------------------------|------------------------|
| AIM density (H)        | Pro-atom density (H)   |


In [ ]:
import glob

# remvoe all cube files
import os

for f in glob.glob("*.cube"):
    os.remove(f)